In [503]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [404]:
#Libraries setup
os.chdir('/dados/home-credit')
print(os.getcwd())
pd.set_option('display.max_columns', 200)
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

/dados/home-credit


In [453]:
#Importing datasets
train = pd.read_csv('application_train.csv')
#test = pd.read_csv('application_test.csv')
#b1 = pd.read_csv('bureau.csv')
#bb2 = pd.read_csv('bureau_balance.csv')
#pa1 = pd.read_csv('previous_application.csv')
#pos2 = pd.read_csv('POS_CASH_balance.csv')
#ip2 = pd.read_csv('installments_payments.csv')
#cb2 = pd.read_csv('credit_card_balance.csv')
description = pd.read_csv("/home/shink/gdata/hcg_gabi_shin/HomeCredit_columns_description.csv", encoding = "ISO-8859-1")
#sample = pd.read_csv('sample_submission.csv')

In [454]:
#dataset_setup
train.columns = map(str.lower, train.columns)
description.columns = map(str.lower, description.columns)
train.head()

sk_id_curr  target name_contract_type code_gender flag_own_car  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  flag_own_realty  cnt_children  amt_income_total  amt_credit  amt_annuity  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

   amt_goods_price name_type_suite name_income_type  \
0         351000.0   Unaccompanied          Working   
1        1129500.0          Family    State servant   
2         135000.0   Unaccompanied          Working   
3         297000.0   Unaccompanied          Working   
4         513000.0   Unaccompanied          Working   

             name_education_type    name_family_status  name_housing_type  \
0  Secondary / secondary special  Single / not married  House / apartment   
1               Higher education               Married  House / apartment   
2  Secondary / secondary special  Single / not married  House / apartment   
3  Secondary / secondary special        Civil marriage  House / apartment   
4  Secondary / secondary special  Single / not married  House / apartment   

   region_population_relative  days_birth  days_employed  days_registration  \
0                    0.018801       -9461           -637            -3648.0   
1                    0.003541      -16765          -1188            -1186.0   
2                    0.010032      -19046           -225            -4260.0   
3                    0.008019      -19005          -3039            -9833.0   
4                    0.028663      -19932          -3038            -4311.0   

   days_id_publish  own_car_age  flag_mobil  flag_emp_phone  flag_work_phone  \
0            -2120          NaN           1               1                0   
1             -291          NaN           1               1                0   
2            -2531         26.0           1               1                1   
3            -2437          NaN           1               1                0   
4            -3458          NaN           1               1                0   

   flag_cont_mobile  flag_phone  flag_email occupation_type  cnt_fam_members  \
0                 1           1           0        Laborers              1.0   
1                 1           1           0      Core staff              2.0   
2                 1           1           0        Laborers              1.0   
3                 1           0           0        Laborers              2.0   
4                 1           0           0      Core staff              1.0   

   region_rating_client  region_rating_client_w_city  \
0                     2                            2   
1                     1                            1   
2                     2                            2   
3                     2                            2   
4                     2                            2   

  weekday_appr_process_start  hour_appr_process_start  \
0                  WEDNESDAY                       10   
1                     MONDAY                       11   
2                     MONDAY                        9   
3                  WEDNESDAY                       17   
4                   THURSDAY                       11   

   reg_region_not_live_region  reg_region_not_work_region  \
0                           0                           0   
1                           0                           0   
2                        

In [455]:
train = train.drop(['target','sk_id_curr'], axis=1)

In [456]:
train = train.sample(n=1000)

In [457]:
    dp = []
    for ls in train.columns:
        if train[ls].isnull().sum()/train.count().max() > 0.3:
            dp.append(ls)
    train.drop(dp, axis=1, inplace=True)

In [458]:
train.head()

name_contract_type code_gender flag_own_car flag_own_realty  \
17349          Cash loans           F            N               N   
256615    Revolving loans           M            N               Y   
89975          Cash loans           F            Y               Y   
91211          Cash loans           F            N               Y   
163199         Cash loans           F            N               N   

        cnt_children  amt_income_total  amt_credit  amt_annuity  \
17349              1          157500.0    614574.0      19822.5   
256615             0          180000.0    180000.0       9000.0   
89975              0          157500.0    290353.5      18684.0   
91211              0          180000.0    283585.5      29187.0   
163199             1           90000.0    284400.0       8752.5   

        amt_goods_price name_type_suite      name_income_type  \
17349          513000.0   Unaccompanied               Working   
256615         180000.0   Unaccompanied         State servant   
89975          220500.0   Unaccompanied               Working   
91211          256500.0          Family  Commercial associate   
163199         225000.0   Unaccompanied  Commercial associate   

                  name_education_type    name_family_status  \
17349   Secondary / secondary special               Married   
256615               Higher education  Single / not married   
89975   Secondary / secondary special               Married   
91211   Secondary / secondary special               Married   
163199  Secondary / secondary special  Single / not married   

          name_housing_type  region_population_relative  days_birth  \
17349   Municipal apartment                    0.018801      -16205   
256615    House / apartment                    0.010643       -8221   
89975     House / apartment                    0.035792      -15840   
91211     House / apartment                    0.007020      -17453   
163199    House / apartment                    0.032561      -12594   

        days_employed  days_registration  days_id_publish  flag_mobil  \
17349           -1993            -8924.0            -5001           1   
256615           -435            -1084.0             -734           1   
89975           -1834            -9092.0            -4060           1   
91211            -258            -9462.0             -982           1   
163199           -953            -2648.0            -1680           1   

        flag_emp_phone  flag_work_phone  flag_cont_mobile  flag_phone  \
17349                1                0                 1           0   
256615               1                0                 1           0   
89975                1                0                 1           0   
91211                1                0                 1           0   
163199               1                0                 1           0   

        flag_email  cnt_fam_members  region_rating_client  \
17349            0              3.0                     2   
256615           0              1.0                     2   
89975            0              2.0                     2   
91211            0              2.0                     2   
163199           0              2.0                     1   

        region_rating_client_w_city weekday_appr_process_start  \
17349                             2                     MONDAY   
256615                            2                     SUNDAY   
89975                             2                   THURSDAY   
91211                             2                   SATURDAY   
163199                            1                     MONDAY   

        hour_appr_process_start  reg_region_not_live_region  \
17349                         7                           0   
256615                       11                           0   
89975                        11                           0   
91211                        10                           0   
163199                       

In [459]:
    for col in train.columns:
        if str(train[col].dtype) == 'float64':
            if train[col].value_counts().count() < 70:
                print(col)
                train[col] = train[col].fillna(train[col].mode().iloc[0]).astype('int64')

cnt_fam_members
obs_30_cnt_social_circle
def_30_cnt_social_circle
obs_60_cnt_social_circle
def_60_cnt_social_circle
amt_req_credit_bureau_hour
amt_req_credit_bureau_day
amt_req_credit_bureau_week
amt_req_credit_bureau_mon
amt_req_credit_bureau_qrt
amt_req_credit_bureau_year


In [460]:
    for col in train.columns:      
        if str(train[col].dtype) == 'int64':
            if train[col].value_counts().count() > 50:
                print(col)
                train[col] = train[col].fillna(train[col].mean()).astype('float64') 

days_birth
days_employed
days_id_publish


In [467]:
    for col in train.columns:              
        if str(train[col].dtype) == 'int64':
            print(col)
            train[col] = train[col].fillna(train[col].mode().iloc[0])
            train[col] = LabelEncoder().fit_transform(train[col])

cnt_children
flag_mobil
flag_emp_phone
flag_work_phone
flag_cont_mobile
flag_phone
flag_email
cnt_fam_members
region_rating_client
region_rating_client_w_city
hour_appr_process_start
reg_region_not_live_region
reg_region_not_work_region
live_region_not_work_region
reg_city_not_live_city
reg_city_not_work_city
live_city_not_work_city
obs_30_cnt_social_circle
def_30_cnt_social_circle
obs_60_cnt_social_circle
def_60_cnt_social_circle
flag_document_2
flag_document_3
flag_document_4
flag_document_5
flag_document_6
flag_document_7
flag_document_8
flag_document_9
flag_document_10
flag_document_11
flag_document_12
flag_document_13
flag_document_14
flag_document_15
flag_document_16
flag_document_17
flag_document_18
flag_document_19
flag_document_20
flag_document_21
amt_req_credit_bureau_hour
amt_req_credit_bureau_day
amt_req_credit_bureau_week
amt_req_credit_bureau_mon
amt_req_credit_bureau_qrt
amt_req_credit_bureau_year


In [470]:
    for col in train.columns:              
        if str(train[col].dtype) == 'object':
            print(col)
            train[col] = train[col].fillna(train[col].mode().iloc[0])
            train[col] = LabelEncoder().fit_transform(train[col])

name_contract_type
code_gender
flag_own_car
flag_own_realty
name_type_suite
name_income_type
name_education_type
name_family_status
name_housing_type
weekday_appr_process_start
organization_type


In [513]:
    nc = []
    for col in train.columns:   
        if str(train[col].dtype) == 'float64':
            print(col)
            train[col] = train[col].fillna(train[col].mean())
            nc.append(col)
            train[col] = Normalizer().fit_transform(train[nc])

amt_income_total
amt_credit
amt_annuity
amt_goods_price
region_population_relative
days_birth
days_employed
days_registration
days_id_publish
ext_source_2
ext_source_3
days_last_phone_change


In [514]:
train.head()

name_contract_type  code_gender  flag_own_car  flag_own_realty  \
17349                    0            0             0                0   
256615                   1            1             0                1   
89975                    0            0             1                1   
91211                    0            0             0                1   
163199                   0            0             0                0   

        cnt_children  amt_income_total  amt_credit  amt_annuity  \
17349              1               1.0    0.000002     0.000050   
256615             0               1.0    0.000006     0.000111   
89975              0               1.0    0.000003     0.000054   
91211              0               1.0    0.000004     0.000034   
163199             1               1.0    0.000004     0.000114   

        amt_goods_price  name_type_suite  name_income_type  \
17349          0.000002                6                 3   
256615         0.000006                6                 2   
89975          0.000005                6                 3   
91211          0.000004                1                 0   
163199         0.000004                6                 0   

        name_education_type  name_family_status  name_housing_type  \
17349                     4                   1                  2   
256615                    1                   3                  1   
89975                     4                   1                  1   
91211                     4                   1                  1   
163199                    4                   3                  1   

        region_population_relative  days_birth  days_employed  \
17349                     0.999823    0.000062       0.000502   
256615                    0.999943    0.000122       0.002299   
89975                     0.999360    0.000063       0.000545   
91211                     0.999975    0.000057       0.003876   
163199                    0.999470    0.000079       0.001049   

        days_registration  days_id_publish  flag_mobil  flag_emp_phone  \
17349            0.000112         0.000200           0               1   
256615           0.000923         0.001362           0               1   
89975            0.000110         0.000246           0               1   
91211            0.000106         0.001018           0               1   
163199           0.000378         0.000595           0               1   

        flag_work_phone  flag_cont_mobile  flag_phone  flag_email  \
17349                 0                 1           0           0   
256615                0                 1           0           0   
89975                 0                 1           0           0   
91211                 0                 1           0           0   
163199                0                 1           0           0   

        cnt_fam_members  region_rating_client  region_rating_client_w_city  \
17349                 2                     1                            1   
256615                0                     1                            1   
89975                 1                     1                            1   
91211                 1                     1                            1   
163199                1                     0                            0   

        weekday_appr_process_start  hour_appr_process_start  \
17349                            1                        4   
256615                           3                        8   
89975                            4                        8   
91211                            2                        7   
163199                           1                       11   

        reg_region_not_live_region  reg_region_not_work_region  \
17349                            0                           0   
256615                           0                           0   
89975                            0                           0   
912

# Tratamento do train

## Categorical

### Categorial without nulls

name_contract_type - 2
code_gender - 3 *
flag_own_car - 2
flag_own_realty - 2
cnt_children - 15 *
name_income_type - 8 *
name_education_type - 5 *
name_family_status - 6 *
name_housing_type - 6 *
flag_emp_phone 
flag_work_phone
flag_cont_mobile
flag_phone
flag_email
occupation_type - 19 *
cnt_fam_members - 18 *
region_rating_client - 3
region_rating_client_w_city - 3
weekday_appr_process_start - 7
hour_appr_process_start - 24
reg_region_not_live_region 
reg_region_not_work_region
live_region_not_work_region 
reg_city_not_live_city 
live_city_not_work_city
organization_type - 59
flag_document_2 até flag_document_21

### Categorical with nulls

name_type_suite - 7 *
fondkapremont_mode - 4
housetype_mode - 3
wallsmaterial_mode - 7
emergencystate_mode - 2
obs_30_cnt_social_circle - 32
def_30_cnt_social_circle - 10
obs_60_cnt_social_circle - 32
def_60_cnt_social_circle - 10
amt_req_credit_bureau_hour - 5
amt_req_credit_bureau_day - 10
amt_req_credit_bureau_week - 9
amt_req_credit_bureau_mon - 27
amt_req_credit_bureau_qrt - 10
amt_req_credit_bureau_year - 25

## Continuous

### Continuous without nulls

amt_income_total
amt_credit
region_population_relative
days_birth
days_employed
days_registration 
days_id_publish
own_car_age - * possivel transformar em categorico
days_last_phone_change

### Continuous with nulls

amt_annuity (poucos nulls)
amt_goods_price
ext_source_1 - normalized
ext_source_2 - normalized
ext_source_3 - normalized
apartments_avg - normalized
basementarea_avg - normalized
years_beginexpluatation_avg - normalized
years_build_avg - normalized
commonarea_avg
elevators_avg
entrances_avg
floorsmax_avg
floorsmax_avg
floorsmin_avg
landarea_avg
livingapartments_avg
livingarea_avg
nonlivingapartments_avg
nonlivingarea_avg
mode - 58 a 72
median - 59 a 85

### Candidato à dropar

flag_mobil